## Unit 16. CNN 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定與資料下載

In [ ]:
from pathlib import Path
import tensorflow as tf
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P4_Unit16_HW'
SOURCE_DATA_DIR = 'P4_Unit16_HW_Data'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")

try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_4'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

# ========================================
# 檢查 GPU 狀態
# ========================================
print(f"\nTensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ 偵測到 GPU：{gpus[0].name}")
    print("  （訓練速度將明顯快於僅用 CPU）")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("△ 未偵測到 GPU。")
    print("  訓練速度將使用 CPU（速度較慢但仍可完成）")

---
### **I. 練習題: 建立一個CNN模型進行 Cats vs Dogs 影像分類**

**數據集資訊**:
- **來源**: Kaggle Cats and Dogs Filtered Dataset
- **下載連結**: https://www.kaggle.com/api/v1/datasets/download/birajsth/cats-and-dogs-filtered
- **數據結構**: 
  - 訓練集: cats_and_dogs_filtered/train/cats/, cats_and_dogs_filtered/train/dogs/
  - 驗證集: cats_and_dogs_filtered/validation/cats/, cats_and_dogs_filtered/validation/dogs/
- **影像格式**: JPG彩色影像，尺寸不一
- **類別**: 2類別（貓、狗）

**學習目標**:
1. 掌握 CNN 模型的基本架構（卷積層、池化層、全連接層）
2. 學習使用 `ImageDataGenerator` 進行數據增強
3. 理解二元分類任務的設計與評估
4. 分析過擬合現象並使用 Dropout 改善

### **1. 數據下載與解壓縮**

**任務**:
- 下載數據集
- 檢查數據集結構，確認訓練集和驗證集的路徑
- 統計每個類別的影像數量

```python
# 檢查數據結構
train_dir = DATA_DIR / 'cats_and_dogs_filtered' / 'train'
validation_dir = DATA_DIR / 'cats_and_dogs_filtered' / 'validation'
```

In [ ]:
# 數據來源: Kaggle Cats and Dogs Filtered Dataset
# https://www.kaggle.com/api/v1/datasets/download/birajsth/cats-and-dogs-filtered

import requests
import os
import zipfile

# 1. 設定路徑與 URL
url = "https://www.kaggle.com/api/v1/datasets/download/birajsth/cats-and-dogs-filtered"
zip_path = os.path.join(DATA_DIR, "cats-and-dogs-filtered.zip")
data_file = os.path.join(DATA_DIR, "cats-and-dogs-filtered.zip")
extract_path = DATA_DIR

def download_and_extract():
    # --- 步驟 A: 下載檔案 ---
    print(f"正在從 {url} 下載...")
    try:
        # allow_redirects=True 處理 Kaggle 的重導向
        response = requests.get(url, allow_redirects=True, stream=True)
        response.raise_for_status() # 若下載失敗會拋出異常
        
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"下載成功，檔案儲存於: {zip_path}")

        # --- 步驟 B: 解壓縮檔案 ---
        if zipfile.is_zipfile(zip_path):
            print(f"正在解壓縮至: {extract_path}...")
            # 確保目標資料夾存在
            if not os.path.exists(extract_path):
                os.makedirs(extract_path)
            
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            
            print("解壓縮完成！")
            
            # (選填) 步驟 C: 刪除原始 ZIP 檔以節省空間
            # os.remove(zip_path)
            # print("已移除原始 ZIP 壓縮檔。")
        else:
            print("錯誤：下載的檔案不是有效的 ZIP 格式。")
            
    except Exception as e:
        print(f"發生錯誤: {e}")

if __name__ == "__main__" and not os.path.exists(data_file):
    download_and_extract()
else:
    print(f"檔案已存在於: {data_file}")

# 檢查數據結構（加入存在性檢查並過濾影像副檔名）
base_dir = os.path.join(extract_path, 'cats_and_dogs_filtered')
if not os.path.exists(base_dir):
    print(f"資料夾不存在: {base_dir}。請確認是否已完成下載與解壓縮。")
else:
    for subset in ['train', 'validation']:
        subset_dir = os.path.join(base_dir, subset)
        print(f"\n子集: {subset}")
        if not os.path.exists(subset_dir):
            print(f"  子集資料夾不存在: {subset_dir}")
            continue
        for category in ['cats', 'dogs']:
            category_dir = os.path.join(subset_dir, category)
            if not os.path.exists(category_dir):
                print(f"  類別資料夾不存在: {category_dir}")
                continue
            # 只計算常見影像格式的檔案
            exts = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
            try:
                num_images = sum(1 for fname in os.listdir(category_dir) if fname.lower().endswith(exts))
            except Exception as e:
                print(f"  讀取 {category_dir} 時發生錯誤: {e}")
                num_images = 0
            print(f"  類別 '{category}' 有 {num_images} 張圖片")

### **2. 數據探索與可視化**

**任務**:
- 隨機顯示至少 12 張貓和狗的影像（6張貓、6張狗）
- 檢查影像的尺寸分布（寬度、高度、通道數）
- 統計訓練集和驗證集的樣本數
- 繪製類別分布圖

**提示**:
- 使用 `PIL` 或 `cv2` 讀取影像
- 使用 `matplotlib` 的 `subplot` 排列顯示
- 影像路徑可用 `glob` 或 `os.listdir` 獲取

### **3. 數據預處理與數據增強**

**任務**:
- 使用 `ImageDataGenerator` 建立訓練與驗證資料生成器
- **訓練集**需進行數據增強:
  - 影像縮放至 [0, 1]（rescale=1./255）
  - 隨機水平翻轉（horizontal_flip）
  - 隨機旋轉（rotation_range=20）
  - 隨機寬度/高度偏移（width_shift_range=0.2, height_shift_range=0.2）
  - 隨機縮放（zoom_range=0.2）
- **驗證集**僅需縮放，不進行增強
- 設定 `target_size=(150, 150)` 或 `(224, 224)`
- 設定 `batch_size=32` 或 `64`
- 設定 `class_mode='binary'`（二元分類）

**提示**:
```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # 添加其他數據增強參數
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
```

### **4. 可視化數據增強效果**

**任務**:
- 從訓練集中取出一張影像
- 使用 `ImageDataGenerator` 生成 9 張增強後的影像
- 使用 `subplot(3, 3)` 排列顯示
- 觀察旋轉、翻轉、偏移、縮放的效果

**提示**:
```python
img = ... # 讀取一張影像
img = img.reshape((1,) + img.shape)

i = 0
for batch in datagen.flow(img, batch_size=1):
    plt.subplot(3, 3, i+1)
    plt.imshow(batch[0])
    i += 1
    if i >= 9:
        break
```

### **5. 建立 CNN 模型**

**任務**:
- 使用 `Sequential` API 建立 CNN 模型
- **建議架構**:
  - 卷積層 1: `Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3))`
  - 池化層 1: `MaxPooling2D(2, 2)`
  - 卷積層 2: `Conv2D(64, (3,3), activation='relu')`
  - 池化層 2: `MaxPooling2D(2, 2)`
  - 卷積層 3: `Conv2D(128, (3,3), activation='relu')`
  - 池化層 3: `MaxPooling2D(2, 2)`
  - 展平層: `Flatten()`
  - 全連接層 1: `Dense(512, activation='relu')`
  - Dropout層: `Dropout(0.5)`
  - 輸出層: `Dense(1, activation='sigmoid')`（二元分類）
- 使用 `model.summary()` 查看模型結構
- 計算總參數量
- 使用 `plot_model()` 繪製模型架構圖

**提示**:
- 二元分類使用 `sigmoid` 激活函數輸出 [0, 1] 的機率
- Dropout 可防止過擬合
- 影像尺寸經過每層 `MaxPooling2D(2, 2)` 會縮小一半

### **6. 模型編譯**

**任務**:
- 設定優化器: `optimizer='adam'`
- 設定損失函數: `loss='binary_crossentropy'`（二元分類）
- 設定評估指標: `metrics=['accuracy']`
- 說明為何選擇這些設定

**思考**:
- 為什麼二元分類使用 `binary_crossentropy` 而非 `categorical_crossentropy`？
- `adam` 優化器的優勢是什麼？

### **7. 設定 Callbacks**

**任務**:
- 設定 `ModelCheckpoint`:
  - 保存最佳模型至 `MODEL_DIR / 'best_cnn_model.h5'`
  - 監控 `val_accuracy`
  - 僅保存最佳權重 (`save_best_only=True`)
- 設定 `EarlyStopping`:
  - 監控 `val_loss`
  - 設定 `patience=5`
  - 恢復最佳權重 (`restore_best_weights=True`)
- （可選）設定 `ReduceLROnPlateau`:
  - 監控 `val_loss`
  - 當驗證損失停止改善時降低學習率

**提示**:
```python
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [
    ModelCheckpoint(...),
    EarlyStopping(...),
]
```

### **8. 模型訓練**

**任務**:
- 使用 `model.fit()` 訓練模型
- 設定 `epochs=30` 或更多
- 使用 `train_generator` 作為訓練數據
- 使用 `validation_generator` 作為驗證數據
- 設定 `callbacks`
- 記錄訓練時間

**提示**:
```python
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=callbacks
)
```

**注意**:
- 訓練可能需要較長時間（GPU約10-20分鐘，CPU可能需要1-2小時）
- 觀察訓練過程中的 loss 和 accuracy 變化

### **9. 訓練過程視覺化**

**任務**:
- 繪製訓練與驗證的 Loss 曲線（同一張圖）
- 繪製訓練與驗證的 Accuracy 曲線（同一張圖）
- 分析是否有過擬合或欠擬合現象:
  - **過擬合**: 訓練準確率持續上升，驗證準確率停滯或下降
  - **欠擬合**: 訓練和驗證準確率都很低
- 標註最佳驗證準確率出現的 epoch

**提示**:
```python
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
```

### **10. 模型評估**

**任務**:
- 載入最佳模型（從 `ModelCheckpoint` 保存的檔案）
- 在驗證集上計算最終的 Loss 和 Accuracy
- 使用 `model.predict()` 獲取預測機率
- 設定閾值（通常 0.5）將機率轉換為類別標籤
- 計算並顯示:
  - **混淆矩陣** (Confusion Matrix)
  - **分類報告** (Classification Report): Precision, Recall, F1-score
  - **ROC-AUC** 曲線
- 分析模型在哪個類別表現較好

**提示**:
```python
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# 獲取真實標籤和預測機率
y_true = validation_generator.classes
y_pred_prob = model.predict(validation_generator)
y_pred = (y_pred_prob > 0.5).astype(int)

# 計算指標
cm = confusion_matrix(y_true, y_pred)
print(classification_report(y_true, y_pred, target_names=['Cat', 'Dog']))
```

### **11. 預測結果可視化**

**任務**:
- 從驗證集中隨機選取 12 張影像
- 使用模型進行預測
- 顯示影像，並標註:
  - 真實標籤（Ground Truth）
  - 預測標籤（Prediction）
  - 預測機率（Confidence）
- 特別標記預測錯誤的樣本（用紅色邊框）

**提示**:
```python
import random

# 隨機選取影像
cat_images = list((validation_dir / 'cats').glob('*.jpg'))
dog_images = list((validation_dir / 'dogs').glob('*.jpg'))

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
for i, ax in enumerate(axes.flat):
    # 載入影像、預測、顯示
    pass
```

### **12. 錯誤案例分析**

**任務**:
- 找出所有預測錯誤的樣本
- 顯示至少 10 張錯誤案例
- 分析模型為何預測錯誤:
  - 影像是否模糊？
  - 是否有遮擋？
  - 是否有多個動物？
  - 角度是否特殊？
- 提出改進建議

**思考**:
- 這些錯誤案例有什麼共同特徵？
- 如何改進模型以減少這些錯誤？

### **13. 特徵圖可視化（Activation Maps）**

**任務**:
- 選擇一張貓或狗的影像
- 可視化第一層卷積層的特徵圖（32個filter）
- 可視化第二層卷積層的特徵圖（64個filter，選取部分顯示）
- 觀察 CNN 學習到的特徵:
  - 淺層：邊緣、紋理
  - 深層：形狀、部件

**提示**:
```python
from tensorflow.keras.models import Model

# 建立輸出中間層的模型
layer_outputs = [layer.output for layer in model.layers[:3]]  # 前3層
activation_model = Model(inputs=model.input, outputs=layer_outputs)

# 獲取激活值
activations = activation_model.predict(img_array)

# 顯示特徵圖
for activation in activations:
    # 繪製...
    pass
```

### **14. 保存模型**

**任務**:
- 將最終模型保存為 `.h5` 格式
- 將最終模型保存為 `SavedModel` 格式（可選）
- 記錄模型的關鍵資訊:
  - 輸入尺寸
  - 總參數量
  - 訓練時間
  - 最佳驗證準確率
  - 測試準確率

---
## II. 額外加分作業 (自由繳交)

**說明**:
- 學生可自由選擇是否繳交加分作業（下週上課前完成 email 電子檔）
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI、問同學、問學長姊，甚至參考以前別人的作業，但一定要自行[吸收][執行][整理]結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容（與他人雷同、原創性低、抄襲比例過高），作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉（提供作業給別人複製的也一樣）
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

---
### **🌟 遷移學習挑戰: 使用 MobileNetV2 提升性能**

**目標**: 使用預訓練的 MobileNetV2 模型進行遷移學習，提升 Cats vs Dogs 分類性能

**任務**:
1. **載入預訓練模型**:
   - 使用 `tf.keras.applications.MobileNetV2`
   - 載入 ImageNet 預訓練權重
   - 凍結 MobileNetV2 的所有層（`trainable=False`）

2. **添加自定義分類頭**:
   - GlobalAveragePooling2D
   - Dense(128, activation='relu')
   - Dropout(0.5)
   - Dense(1, activation='sigmoid')

3. **兩階段訓練**:
   - **Stage 1 (特徵提取)**: 
     - 凍結 MobileNetV2，僅訓練分類頭
     - 訓練 10 epochs
   - **Stage 2 (微調)**:
     - 解凍 MobileNetV2 的後半部分層（如最後 50 層）
     - 使用較小學習率（如 1e-5）繼續訓練
     - 訓練 10 epochs

4. **性能對比**:
   - 比較從頭訓練的 CNN vs. MobileNetV2 遷移學習
   - 製作對比表格:
     | 指標 | 從頭訓練 CNN | MobileNetV2 | 提升幅度 |
     |-----|------------|-------------|---------|
     | 準確率 | ? | ? | ? |
     | 訓練時間 | ? | ? | ? |
     | 參數量 | ? | ? | ? |

5. **分析**:
   - 為什麼遷移學習效果更好？
   - ImageNet 預訓練的知識如何幫助 Cats vs Dogs 分類？
   - 兩階段訓練的必要性是什麼？

**提示**:
```python
from tensorflow.keras.applications import MobileNetV2

# Stage 1: 特徵提取
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # 凍結

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 編譯與訓練...

# Stage 2: 微調
base_model.trainable = True
# 凍結前面的層，僅微調後面的層
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # 小學習率
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 繼續訓練...
```

**評分標準**:
- 成功實作兩階段訓練: 0.3 分
- 性能提升明顯（準確率 > 95%）: 0.3 分
- 詳細分析與可視化: 0.2 分
- 錯誤案例改善分析: 0.2 分

### 💭 思考題

1. 為什麼 CNN 比 MLP 更適合影像分類任務？
2. 卷積層的 filter 數量（如 32, 64, 128）是如何決定的？
3. MaxPooling 的作用是什麼？為什麼要降採樣？
4. Dropout 在訓練和測試時的行為有何不同？
5. 數據增強為何能防止過擬合？它如何擴展數據集？
6. 為什麼二元分類使用 `binary_crossentropy` 而非 `categorical_crossentropy`？
7. 遷移學習為何有效？ImageNet 的知識如何遷移到 Cats vs Dogs？
8. 兩階段訓練（凍結→微調）的必要性是什麼？
9. 如果驗證準確率在 60-70% 徘徊，可能是什麼原因？如何改進？
10. 在工業應用中，如何選擇合適的影像解析度（如 150×150 vs 224×224）？

---
### 📊 作業完成檢查清單

請在完成後打勾：

**Part I: CNN 基礎練習**
- [ ] 1. 數據下載與解壓縮
- [ ] 2. 數據探索與可視化
- [ ] 3. 數據預處理與數據增強
- [ ] 4. 可視化數據增強效果
- [ ] 5. 建立 CNN 模型
- [ ] 6. 模型編譯
- [ ] 7. 設定 Callbacks
- [ ] 8. 模型訓練
- [ ] 9. 訓練過程視覺化
- [ ] 10. 模型評估
- [ ] 11. 預測結果可視化
- [ ] 12. 錯誤案例分析
- [ ] 13. 特徵圖可視化
- [ ] 14. 保存模型

**Part II: 加分作業（可選）**
- [ ] 遷移學習: MobileNetV2 兩階段訓練
- [ ] 性能對比分析
- [ ] 錯誤案例改善分析

**思考題**
- [ ] 完成至少 5 個思考題的回答

---
# 想對老師說的話